In [ ]:
def asp_try(asp_file, instance):
    solution = ""
    models = None
    success = 0
    try:
        ctl = Control(arguments=["--opt-mode=opt"])
        ctl.load(asp_file)
        ctl.load(instance)
        ctl.add("base", [], solution)
        parts = [("base", [])]
        ctl.ground(parts)      
        try:       
            with ctl.solve(yield_=True, async_=True) as handle: 
                handle.wait(10)
                handle.cancel()
                models = list(iter(handle))
                if len(models) > 0:
                    success = 1
                    #for m in models:
                    #    print(m.symbols(True,True,True))
            if ctl.statistics['solving']['conflicts'] > 0:
                print("Errors occurred during solving.")
                # Analyze the conflicts to get more information
                conflicts_analyzed = ctl.statistics['solving']['conflicts_analyzed']
                print("Conflicts analyzed: {}".format(conflicts_analyzed))
            else:
                print("No errors occurred during solving.")
        except:
            print("Unsatisfiable")
    except:
        print("Parsing or Safety Issues")
    return success, models

In [ ]:
def asp_try_v2(asp_file, instance):
    
    input_files = [asp_file, instance]
    asp_program = []

    def custom_logger(code, message):
        if code == clingo.MessageCode.Warning:
            print(f"Warning: {message}")
        elif code == clingo.MessageCode.Error:
            print(f"Error: {message}")
        elif code == clingo.MessageCode.Info:
            print(f"Info: {message}")
        elif code == clingo.MessageCode.Debug:
            print(f"Debug: {message}")

    # Create a Control object with the custom logger
    control = clingo.Control(logger=custom_logger,message_limit=100)
    print("debug")
    def on_model(model):
        print("Model:", model)
        
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    control.add("base", [], "".join(asp_program))
    print("debug2")
    
    control.ground([("base", [])])
    print("debug3")
    #except Exception as e:
    #    error_message = control.messages
    #    print(error_message)
    #    print(e)
    #    print("Grounding Error")
    #    return None
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
        print("No errors occurred during solving.")
    except Exception as e:
        print(e)
        #if control.statistics['solving']['solvers']['conflicts'] > 0:
        print("Errors occurred during solving.")
        # Analyze the conflicts to get more information
        conflicts_analyzed = control.statistics['solving'] #['conflicts_analyzed']
        print("Conflicts analyzed: {}".format(conflicts_analyzed))
        #else:            

In [ ]:
def asp_try_3(asp_file,instance):
    errors = []
    messages = []
    def custom_logger(code, message):
        errors.append(code)
        messages.append(message)
    control = clingo.Control(logger=custom_logger,message_limit=100)
    input_files = [asp_file, instance]
    asp_program = []
    models = []
    def on_model(model):
        print("Model:", model)
        models.append(model)
    
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    control.add("base", [], "".join(asp_program))
    try:
        control.ground([("base", [])])
    except:
        print("Grounding stopped because of errors")
        error_codes = ["AtomUndefined","FileIncluded","GlobalVariable","OperationUndefined","Other","RuntimeError","VariableUnbounded"]
        sub_codes = ["unsafe"]
        error_dict = {}
        for error_code in error_codes:
            error_dict[error_code] = 0
        sub_dict = {}
        for sub_code in sub_codes:
            sub_dict[sub_code] = 0
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    sub_dict[sc] += 1
        return models, error_dict, message_dict
        
    control.configuration.solve.models = 0  # Limit the number of models to 1
    control.solve(on_model=on_model)
    
    return models, error_dict, message_dict

In [ ]:
def asp_try_3(asp_file,instance):    
    errors = []
    #messages = []
    models = []  
    symbols = []
    
    def custom_logger(code, message):
        errors.append(code)
        messages.append(message)

    #def on_model(model):
    #    #print("Model:", model)
    # #   models.append(model)
    #    with open('output.txt','w') as file:
    #        print(model, file=file)
    #    with open('output.txt','r') as file:
    #        lines = file.readlines()
    #        symbols_list = lines[0].split()
    #        for s in symbols_list:
    #            symbols.append(s)
    #    os.remove("output.txt")
        
    #error_codes = ["AtomUndefined","FileIncluded","GlobalVariable","OperationUndefined","Other","RuntimeError","VariableUnbounded"]
    #sub_codes = ["unsafe"]
    asp_program = []  
    #error_dict = {}
    #message_dict = {}
    #for error_code in error_codes:
    #    error_dict[error_code] = 0
    #for sub_code in sub_codes:
    #   message_dict[sub_code] = 0

    ##clingo_control = Control(['0', '--warn=none', '--opt-mode=optN', '-t', '4'])
    control = clingo.Control(['0', '--warn=none', '--opt-mode=optN', '-t', '4'],logger=custom_logger,message_limit=100)
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        print(f"Unexpected {err=}, {type(err)=}.")
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    message_dict[sc] += 1
        return models, error_dict, message_dict, errors, messages, symbols
    try:
        control.ground([("base", [])])
    except Exception as err:         
        print(f"Unexpected {err=}, {type(err)=}.")
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    message_dict[sc] += 1
        return models, error_dict, message_dict, errors, messages, symbols
        
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        print(f"Unexpected {err=}, {type(err)=}.")
        for e in errors:
            for ec in error_codes:
                if str(e).split(".")[1] == ec:
                    error_dict[ec] += 1
        for mss in messages:
            for sc in sub_codes:
                if sc in mss:
                    message_dict[sc] += 1
        return models, error_dict, message_dict, errors, messages, symbols
    
    return models, error_dict, message_dict, errors, messages, symbols

In [ ]:
def gen_answer_set(program, opt=False):
        """
        Args:
            program (str): a string of ASP program
            opt (bool): if true, only optimal answer sets are returned
                        leave it to False when there is no weak constraint
        """
        clingo_control = Control(['0', '--warn=none', '--opt-mode=optN', '-t', '4'])
        models = []
        try:
            clingo_control.add('base', [], program)
            clingo_control.ground([('base', [])], context=Context())
        except:
            # breakpoint()
            return []
        if opt:
            clingo_control.solve(on_model = lambda model: models.append(model.symbols(atoms=True)) if model.optimality_proven else None)
        else:
            clingo_control.solve(on_model = lambda model: models.append(model.symbols(atoms=True)))
        models = [[str(atom) for atom in model] for model in models]
        return models

In [ ]:
def asp_try_4(asp_file,instance):
    models = []  
    errors = []
    symbols = []

    def on_model(model):
        #print("Model:", model)
        models.append(model)
        with open('output.txt','w') as file:
            print(model, file=file)
        with open('output.txt','r') as file:
            lines = file.readlines()
            symbols_list = lines[0].split()
            #for s in symbols_list:
            symbols.append(symbols_list)
        os.remove("output.txt")

    time_init = time.time() 
    asp_program = []
    # TODO: timeout of Unix systems. Wrapper around. Kill the process. 
    control = clingo.Control(['--warn=none','-t','5','-n','10']) #, '--warn=none', '--opt-mode=optN' --time-limit=5, -t 5
    input_files = [asp_file, instance]
    for file_name in input_files:
        with open(file_name, "r") as file:
            asp_program.extend(file.readlines())
            
    try:
        control.add("base", [], "".join(asp_program))
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols
    
    try:
        control.ground([("base", [])])
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols
    
    #control.configuration.solve.models = 0  # Limit the number of models to 1
    try:
        control.solve(on_model=on_model)
    except Exception as err:         
        #print(f"Unexpected {err=}, {type(err)=}.")
        errors.append(str(err))
        return models, errors, symbols
    
    time_final = time.time() 
    delta_time = time_final-time_init
    
    if delta_time > 5 and len(symbols) == 0:
        errors.append("Timeout")

    #print(f"Results: {models}, {errors}, {symbols}. Timedelta {delta_time}")
    return models, errors, symbols